In [1]:
import pandas as pd
import numpy as np
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from category_encoders.leave_one_out import LeaveOneOutEncoder
import re
import warnings
from sklearn.tree import DecisionTreeClassifier

warnings.simplefilter("ignore")
%matplotlib inline


## Обработаем тестовую выборку ##

In [18]:
df_test = pd.read_csv('test.csv')

In [19]:
df_test

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a


In [20]:
df_test.isnull().sum()

id                            0
cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [21]:
columns = []
for col in df_test:
    if df_test[col].isnull().sum() <100 and col != 'id':
        columns.append(col)

In [22]:
columns

['cap-diameter',
 'cap-shape',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'habitat',
 'season']

In [23]:
df_copy_test_ = pd.DataFrame()

In [24]:
for i in columns:
    if i in df_test.columns and df_test[i].dtypes == 'float64':        
        df_copy_test_[i]=df_test[i].fillna(df_test[i].mean())
    else:
#     elif i in df_test.columns and df_test[i].dtypes == 'object':
        df_copy_test_[i]=df_test[i].fillna(df_test[i].mode()[0])

In [25]:
df_copy_test_.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [26]:
df_copy_test_.head()

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,8.64,x,n,t,w,11.13,17.12,w,t,d,a
1,6.90,o,o,f,y,1.27,10.75,n,f,d,a
2,2.00,b,n,f,n,6.18,3.14,n,f,d,s
3,3.47,x,n,f,n,4.98,8.51,w,t,d,u
4,6.17,x,y,f,y,6.73,13.70,y,t,d,u


In [27]:
df_copy_test_.isnull().sum()

cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
habitat                 0
season                  0
dtype: int64

In [28]:
df_copy_test_.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [29]:
for column in columns:
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    for i,value in enumerate(df_copy_test_[column]):
        #print(value)
        #print(value)
           
        value = value.replace(" ","")
        
        if re.match("^\d+?\.\d+?$", value) or re.match("^\d+?$", value):
            #print(f"число: {value}") 
            #print(f"мода: {df_copy_test_['cap-shape'].mode()[0]}")
            df_copy_test_[column].loc[i] = df_copy_test_[column].mode()[0]
        elif len(value) > 1:        
            #print(f"не число а строка: {value}:")
            #print(f"не число а строка,мода: {df_copy_test_['cap-shape'].mode()[0]}")
            #i+=1
            df_copy_test_[column].loc[i] = df_copy_test_[column].mode()[0]

        else:
           #print(f"не число: {value}")
           #i = i.append(value)
            pass

#print(i)
    #print(f"{value}: \n{type(value)}")
    

In [86]:
encoder = OneHotEncoder(return_df=True)
X_test = encoder.fit_transform(df_copy_test_)

In [87]:
X_test.shape

(2077964, 162)

## Обработаем тренировочные данные ##

In [32]:
df_train = pd.read_csv('train.csv')

In [33]:
df_train_1 = df_train.copy()

In [34]:
df_train_1.dtypes

id                        int64
class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [17]:
#df_train_1 = df_train.iloc[:1000000]

In [35]:
df_train_1.isnull().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [36]:
df_train_copy = pd.DataFrame()

In [37]:
for column in df_train_1.columns:
    if column == 'class':
        df_train_copy[column] = df_train_1[column]
    if column in columns and df_train_1[column].dtypes == 'float64':
        df_train_copy[column]=df_train_1[column].fillna(df_train_1[column].mean())
    if column in columns and df_train_1[column].dtypes == 'object':
        df_train_copy[column]=df_train_1[column].fillna(df_train_1[column].mode()[0])

In [38]:
df_train_copy.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [39]:
df_train_copy.isnull().sum()

class                   0
cap-diameter            0
cap-shape               0
cap-color               0
does-bruise-or-bleed    0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
habitat                 0
season                  0
dtype: int64

In [40]:
df_train_copy.head()

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,e,8.80,f,u,f,w,4.51,15.39,w,f,d,a
1,p,4.51,x,o,f,n,4.79,6.48,o,t,d,w
2,e,6.94,f,b,f,w,6.85,9.93,n,f,l,w
3,e,3.88,f,g,f,g,4.16,6.53,w,f,d,u
4,e,5.85,x,w,f,w,3.37,8.36,w,f,g,a


In [41]:
df_train_copy['season'].unique()

array(['a', 'w', 'u', 's'], dtype=object)

In [42]:
df_train_copy.shape

(3116945, 12)

In [43]:
for column in columns:
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    for i,value in enumerate(df_train_copy[column]):
        #print(value)
        #print(value)

        value = value.replace(" ","")

        if re.match("^\d+?\.\d+?$", value) or re.match("^\d+?$", value):
            #print(f"число: {value}") 
            #print(f"мода: {df_copy_test_['cap-shape'].mode()[0]}")
            df_train_copy[column].loc[i] = df_train_copy[column].mode()[0]
        elif len(value) > 1:
            #print(f"не число а строка: {value}:")
            #print(f"не число а строка,мода: {df_copy_test_['cap-shape'].mode()[0]}")
            #i+=1
            df_train_copy[column].loc[i] = df_train_copy[column].mode()[0]

        else:
            #print(f"не число: {value}")
            #i = i.append(value)
            pass

#print(i)
#print(f"{value}: \n{type(value)}") 

In [44]:
y = df_train_copy["class"]
y = y.map({"p":-1, "e":1})

In [45]:
X = df_train_copy.drop(["class"], axis=1)

In [46]:
X.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [62]:
# encoder = LeaveOneOutEncoder(return_df=True)
# X2 = encoder.fit_transform(X,y)
# X2.dtypes

cap-diameter            float64
cap-shape               float64
cap-color               float64
does-bruise-or-bleed    float64
gill-color              float64
stem-height             float64
stem-width              float64
stem-color              float64
has-ring                float64
habitat                 float64
season                  float64
dtype: object

In [47]:
encoder = OneHotEncoder(return_df=True)
X_train = encoder.fit_transform(X,y)

In [76]:
encoder = OneHotEncoder(return_df=True)
X2 = encoder.fit_transform(X)

In [48]:
X_train.shape

(3116945, 165)

In [49]:
X_test.shape

(2077964, 164)

## Посмотрим разницу в уникальных данных по столбцам в наших датасетах ##

In [50]:
train_unique = pd.DataFrame()

In [51]:
for i,column in enumerate(columns):
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    train_unique = pd.concat([train_unique,pd.DataFrame(df_train_copy[column].unique())], ignore_index=True, axis=1)    
     

In [52]:
train_unique.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
7    object
dtype: object

In [53]:
train_unique.sort_values(by=5, ascending=True, inplace=True)

In [54]:
train_unique

,0,1,2,3,4,5,6,7
20,i,m,e,l,c,a,o,NaN
5,c,n,b,f,u,c,m,NaN
14,g,d,c,z,z,d,b,NaN
6,s,e,x,p,p,e,u,NaN
0,f,u,f,w,w,f,d,a
7,d,y,p,o,f,g,w,NaN
2,p,b,d,g,n,h,g,u
18,u,c,k,c,s,i,r,NaN
12,l,i,l,d,b,k,k,NaN
8,e,r,g,b,g,l,y,NaN


## Посмотрим на уникальные данные в тестовой выборке ##

In [55]:
test_unique = pd.DataFrame()

In [56]:
for i,column in enumerate(columns):
    if column == 'cap-diameter' or column =='stem-height' or column =='stem-width':
        continue
    test_unique = pd.concat([test_unique,pd.DataFrame(df_copy_test_[column].unique())], ignore_index=True, axis=1)

In [57]:
test_unique.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
7    object
dtype: object

In [58]:
test_unique.sort_values(by=5, ascending=True, inplace=True)

In [59]:
test_unique

,0,1,2,3,4,5,6,7
3,f,k,k,e,g,a,g,w
19,l,c,h,i,c,b,r,NaN
4,s,b,x,f,o,c,h,NaN
13,w,f,e,d,f,d,o,NaN
14,d,s,r,t,h,e,e,NaN
1,o,o,f,y,n,f,p,s
6,c,e,a,g,u,g,l,NaN
18,r,x,g,c,m,h,y,NaN
7,g,g,p,p,e,i,w,NaN
16,u,i,y,a,a,k,c,NaN


In [60]:
columns

['cap-diameter',
 'cap-shape',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'habitat',
 'season']

In [61]:
df_copy_test_.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

## Заменим уникальные значения в тестовых и тренировочных выборках на медианные по столбцам ##

In [62]:
df_test_copy = df_copy_test_

In [63]:
df_test_copy

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,8.64,x,n,t,w,11.13,17.12,w,t,d,a
1,6.90,o,o,f,y,1.27,10.75,n,f,d,a
2,2.00,b,n,f,n,6.18,3.14,n,f,d,s
3,3.47,x,n,f,n,4.98,8.51,w,t,d,u
4,6.17,x,y,f,y,6.73,13.70,y,t,d,u
...,...,...,...,...,...,...,...,...,...,...,...
2077959,0.88,x,w,f,w,2.67,1.35,e,f,d,u
2077960,3.12,x,w,f,w,2.69,7.38,w,f,g,a
2077961,5.73,x,e,f,w,6.16,9.74,y,t,d,a
2077962,5.03,b,n,f,g,6.00,3.46,g,f,d,a


In [64]:
type(df_test_copy)

pandas.core.frame.DataFrame

In [65]:
type(df_copy_test_)

pandas.core.frame.DataFrame

In [66]:
df_copy_test_['has-ring'] = df_copy_test_['has-ring'].replace('b', df_copy_test_['has-ring'].mode()[0])

In [67]:
df_copy_test_.loc[df_copy_test_['has-ring'] == 'b']

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [68]:
df_copy_test_['does-bruise-or-bleed'] = df_copy_test_['does-bruise-or-bleed'].replace('m', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [69]:
df_copy_test_.loc[df_copy_test_['does-bruise-or-bleed'] == 'm']

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [70]:
df_train_copy.dtypes

class                    object
cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [71]:
df_train_copy['does-bruise-or-bleed'] = df_train_copy['does-bruise-or-bleed'].replace('d', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [72]:
df_train_copy.loc[df_train_copy['does-bruise-or-bleed'] == 'd']

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [73]:
df_train_copy['does-bruise-or-bleed'] = df_train_copy['does-bruise-or-bleed'].replace('u', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [74]:
df_train_copy.loc[df_train_copy['does-bruise-or-bleed'] == 'u']

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


In [75]:
df_train_copy['does-bruise-or-bleed'] = df_train_copy['does-bruise-or-bleed'].replace('z', df_copy_test_['does-bruise-or-bleed'].mode()[0])

In [76]:
df_train_copy.loc[df_train_copy['does-bruise-or-bleed'] == 'z']

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season


## Закодируем наши данные 

**Обновим датасеты с данными**

In [91]:
encoder = OneHotEncoder(return_df=True)        # Тестовый датасет
X_test = encoder.fit_transform(df_copy_test_) 

In [92]:
X_test.shape

(2077964, 162)

In [109]:
y_train = df_train_copy["class"]                  # Таргет тренировочного датасета
y_train = y_train.map({"p":-1, "e":1})

In [110]:
X_train = df_train_copy.drop(["class"], axis=1)   # Тренировочный датасет

In [111]:
X_train.shape

(3116945, 11)

In [112]:
encoder = OneHotEncoder(return_df=True)
X_train_2 = encoder.fit_transform(X_train,y_train)

In [106]:
X_test.shape

(2077964, 162)

In [113]:
X_train_2.shape

(3116945, 162)

## Разобьем данные и обучим модель ##

In [114]:
X_train_2_2 = X_train_2[:2000000]

In [115]:
y_train_2_2 = y_train[:2000000]

In [116]:
X_train_test = X_train_2[2000000:]

In [117]:
y_train_test = y_train[2000000:]

In [118]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train_2_2, y_train_2_2, test_size=0.3, random_state=42)

In [119]:
dt =  DecisionTreeClassifier(criterion='entropy', max_depth=30,random_state=18)

In [120]:
dt.fit(Xtrain, ytrain)

y_train_dt = dt.predict(Xtrain)
y_test_dt = dt.predict(Xtest)

In [121]:
print(f"Decision tree accuracy train: {accuracy_score(y_train_dt, ytrain):.8f}")
print(f"Decision tree accuracy test: {accuracy_score(y_test_dt, ytest):.8f}")

Decision tree accuracy train: 0.99476000
Decision tree accuracy test: 0.97013667


In [122]:
y_train_test_dt = dt.predict(X_train_test)

In [123]:
print(f"Decision tree accuracy train: {accuracy_score(y_train_test_dt, y_train_test ):.8f}")

Decision tree accuracy train: 0.97020265


In [150]:
y_test_dt = dt.predict(X2_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- cap-color_24
- cap-color_25
- cap-color_26
- cap-color_27
- cap-color_28
- ...


In [ ]:
dt =  DecisionTreeClassifier(criterion='entropy', max_depth=30,random_state=18)

In [124]:
from sklearn.model_selection import GridSearchCV

In [125]:
gscv_rf = GridSearchCV(DecisionTreeClassifier(),
                       {'criterion' : ['gini', 'entropy'],
                        'max_depth': range(15,40)},
                       )

In [126]:
%%time

gscv_rf.fit(X_train_2,y_train)
print(gscv_rf.best_score_)

0.9735930534545844
CPU times: user 4h 46min 53s, sys: 14min 41s, total: 5h 1min 35s
Wall time: 5h 1min 38s


In [115]:
y_train

0         1
1        -1
2         1
3         1
4         1
         ..
999995   -1
999996    1
999997   -1
999998    1
999999    1
Name: class, Length: 1000000, dtype: int64

In [128]:
gscv_rf.best_params_

{'criterion': 'entropy', 'max_depth': 24}